In [1]:
# Colab cell
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
# Adjust these two for YOUR repo
REPO_OWNER = "ywanglab"
REPO_NAME  = "STAT4160"   # e.g., unified-stocks-team1
BASE_DIR   = "/content/drive/MyDrive/dspt25"
CLONE_DIR  = f"{BASE_DIR}/{REPO_NAME}"
REPO_URL   = f"https://github.com/{REPO_OWNER}/{REPO_NAME}.git"

# if on my office computer

# REPO_NAME  = "lectureNotes"   # e.g., on my office computer
# BASE_DIR = r"E:\OneDrive - Auburn University Montgomery\teaching\AUM\STAT 4160 Productivity Tools" # on my office computer
# CLONE_DIR  = f"{BASE_DIR}\{REPO_NAME}"

import os, pathlib
pathlib.Path(BASE_DIR).mkdir(parents=True, exist_ok=True)


In [4]:
import os, subprocess, shutil, pathlib

if not pathlib.Path(CLONE_DIR).exists():
    !git clone {REPO_URL} {CLONE_DIR}
else:
    # If the folder exists, just ensure it's a git repo and pull latest
    os.chdir(CLONE_DIR)
    # !git status
    # !git pull --rebase # !git pull --ff-only
os.chdir(CLONE_DIR)
print("Working dir:", os.getcwd())

Working dir: /content/drive/MyDrive/dspt25/STAT4160



```python
con.executemany(
    "INSERT OR IGNORE INTO meta(ticker,name,sector) VALUES(?,?,?)",
    meta.itertuples(index=False, name=None)
)
```

1. **`con.executemany(...)`**
   Runs the given SQL statement multiple times, once for each set of parameters in the iterable you pass.

2. **SQL part:**

   ```sql
   INSERT OR IGNORE INTO meta(ticker, name, sector) VALUES (?, ?, ?)
   ```

   * `INSERT` → tries to insert a new row.
   * `OR IGNORE` → if the row would violate a constraint (e.g. `ticker` already exists since it’s the primary key), SQLite skips that row instead of raising an error.
   * `?, ?, ?` → placeholders for the three values.

3. **`meta.itertuples(index=False, name=None)`**

   * Iterates over the rows of the pandas DataFrame `meta`.
   * Each row is returned as a plain tuple (since `name=None`).
   * `index=False` means the DataFrame index isn’t included in the tuple.
   * Example: a row like

     | ticker | name       | sector |
     | ------ | ---------- | ------ |
     | AAPL   | Apple Inc. | Tech   |

     becomes the tuple:

     ```python
     ('AAPL', 'Apple Inc.', 'Tech')
     ```

4. **Together:**
   For each row in `meta`, SQLite substitutes the tuple into `(?,?,?)` and attempts to insert it into the `meta` table.
   If the `ticker` already exists, the `IGNORE` clause prevents duplication.


```python
pd.to_datetime(df["date"]).dt.strftime("%Y-%m-%d")
```

1. **`df["date"]`**

   * Selects the `"date"` column from the DataFrame `df`.
   * This column contains date strings in various formats, e.g. `"09/19/2025"`, `"2025.09.20"`, `"2025-09-21"`, etc.
2. **`pd.to_datetime(df["date"])`**

   * Converts that column into pandas **datetime objects**.
   * Handles many formats automatically, so `"2025/09/19"` → `Timestamp('2025-09-19 00:00:00')`.

3. **`.dt` accessor**

   * Gives you access to datetime-specific methods and attributes for each element (like `.year`, `.month`, `.weekday`, etc.).

4. **`.strftime("%Y-%m-%d")`**

   * Formats each datetime as a string, following the given format code:

     * `%Y` → 4-digit year (e.g. 2025)
     * `%m` → 2-digit month (01–12)
     * `%d` → 2-digit day (01–31)
   * Example: `Timestamp("2025-09-19")` → `"2025-09-19"`.

This line is registering a **custom SQL function** with your SQLite connection. Let’s unpack it:

```python
con.create_function("SQRT", 1, lambda x: math.sqrt(x) if x is not None and x >= 0 else None)
```


1. **`con.create_function(name, num_params, func)`**

   * `name` → the SQL function name you want to add (here `"SQRT"`).
   * `num_params` → how many arguments this function takes (here `1`).
   * `func` → a Python function (or lambda) that does the actual work.

   This tells SQLite: *“When I see `SQRT(...)` in a query, call this Python function.”*

---

2. **The lambda function:**

   ```python
   lambda x: math.sqrt(x) if x is not None and x >= 0 else None
   ```

   * If `x` is `None` or negative → return `None` (SQLite will store it as NULL).
   * Otherwise → return the square root using Python’s `math.sqrt`.

---

3. **Result:**
   You can now use `SQRT` inside SQL queries just like a built-in function:

   ```python
   import math, sqlite3
   con = sqlite3.connect(":memory:")
   con.create_function("SQRT", 1, lambda x: math.sqrt(x) if x is not None and x >= 0 else None)

   cur = con.cursor()
   cur.execute("SELECT SQRT(16), SQRT(2), SQRT(-1), SQRT(NULL)").fetchall()
   ```

   Output:

   ```
   [(4.0, 1.4142135623730951, None, None)]
   ```



In [5]:
import sqlite3, math
import pandas as pd
import numpy as np
# Ensure DB exists (fallback: build from CSV)
db_path = pathlib.Path("data/prices.db")
if not db_path.exists():
    print("prices.db not found; attempting minimal build from data/raw/prices.csv …")
    pathlib.Path("data").mkdir(exist_ok=True)
    con = sqlite3.connect(db_path)
    con.execute("PRAGMA foreign_keys = ON;")
    con.executescript("""
    CREATE TABLE IF NOT EXISTS meta (
      ticker TEXT PRIMARY KEY,
      name   TEXT,
      sector TEXT NOT NULL
    );
    CREATE TABLE IF NOT EXISTS prices (
      ticker     TEXT NOT NULL,
      date       TEXT NOT NULL,
      adj_close  REAL NOT NULL CHECK (adj_close >= 0),
      volume     INTEGER NOT NULL CHECK (volume >= 0),
      log_return REAL NOT NULL,
      PRIMARY KEY (ticker,date),
      FOREIGN KEY (ticker) REFERENCES meta(ticker)
    );
    CREATE INDEX IF NOT EXISTS idx_prices_date ON prices(date);
    """)
    # Minimal meta from tickers_25 or from CSV
    if pathlib.Path("tickers_25.csv").exists():
        tks = pd.read_csv("tickers_25.csv")["ticker"].dropna().unique().tolist()
    else:
        raw = pd.read_csv("data/raw/prices.csv")
        tks = raw["ticker"].dropna().unique().tolist()
    meta = pd.DataFrame({"ticker": tks, "name": tks, "sector": ["Unknown"]*len(tks)})
    con.executemany("INSERT OR IGNORE INTO meta(ticker,name,sector) VALUES(?,?,?)",
                    meta.itertuples(index=False, name=None))
    # Load prices.csv if present; otherwise synthesize small sample
    if pathlib.Path("data/raw/prices.csv").exists():
        df = pd.read_csv("data/raw/prices.csv", parse_dates=["date"]).copy()
    else:
        dates = pd.bdate_range("2022-01-03", periods=90)
        rng = np.random.default_rng(7)
        frames=[]
        for t in tks[:5]:
            r = rng.normal(0, 0.01, len(dates))
            price = 100*np.exp(np.cumsum(r))
            vol = rng.integers(1e5, 5e6, len(dates))
            frames.append(pd.DataFrame({"ticker": t, "date": dates,
                                        "adj_close": price, "volume": vol}))
        df = pd.concat(frames, ignore_index=True)
        df["log_return"] = np.log(df["adj_close"]).diff().fillna(0)
    df["date"] = pd.to_datetime(df["date"]).dt.strftime("%Y-%m-%d")
    df = df[["ticker","date","adj_close","volume","log_return"]].drop_duplicates(["ticker","date"])
    con.executemany("INSERT OR REPLACE INTO prices(ticker,date,adj_close,volume,log_return) VALUES(?,?,?,?,?)",
                    df.itertuples(index=False, name=None))
    con.commit()
    con.close()

# Connect and register SQRT (SQLite lacks STDDEV; we’ll compute var and take sqrt)
con = sqlite3.connect(db_path)
con.create_function("SQRT", 1, lambda x: math.sqrt(x) if x is not None and x>=0 else None)
print("SQLite version:", sqlite3.sqlite_version)

SQLite version: 3.37.2


```sql
LAG(log_return,1) OVER (PARTITION BY ticker ORDER BY date) AS lag1,
LAG(log_return,2) OVER (PARTITION BY ticker ORDER BY date) AS lag2,
LEAD(log_return,1) OVER (PARTITION BY ticker ORDER BY date) AS r_tplus1
```

---

### 1. Window functions: `LAG` and `LEAD`

* **`LAG(column, offset)`** → looks *backward* in the ordered rows and returns the value of that column from `offset` rows earlier.
* **`LEAD(column, offset)`** → looks *forward* in the ordered rows and returns the value from `offset` rows later.

If there’s no such row (e.g. the first row doesn’t have a lag, the last row doesn’t have a lead), the result is `NULL`.

---

### 2. `OVER (PARTITION BY ticker ORDER BY date)`

* **`PARTITION BY ticker`** → resets the window for each `ticker`.
  So calculations are done separately for each stock (or entity).
* **`ORDER BY date`** → ensures rows are processed in chronological order for each ticker.

---

### 3. Aliases

* **`lag1`** = yesterday’s log return (`t-1`) for that ticker.
* **`lag2`** = the log return from two days ago (`t-2`).
* **`r_tplus1`** = tomorrow’s log return (`t+1`), useful as a *label* in predictive modeling.

---

### 4. Example

Suppose table `prices` has:

| ticker | date       | log\_return |
| ------ | ---------- | ----------- |
| AAPL   | 2025-09-16 | 0.01        |
| AAPL   | 2025-09-17 | -0.02       |
| AAPL   | 2025-09-18 | 0.03        |
| AAPL   | 2025-09-19 | 0.04        |

Then the query would produce:

| ticker | date       | log\_return | lag1  | lag2  | r\_tplus1 |
| ------ | ---------- | ----------- | ----- | ----- | --------- |
| AAPL   | 2025-09-16 | 0.01        | NULL  | NULL  | -0.02     |
| AAPL   | 2025-09-17 | -0.02       | 0.01  | NULL  | 0.03      |
| AAPL   | 2025-09-18 | 0.03        | -0.02 | 0.01  | 0.04      |
| AAPL   | 2025-09-19 | 0.04        | 0.03  | -0.02 | NULL      |

---



In [6]:
import pandas as pd

sql = """
SELECT ticker, date,
       log_return AS r,
       LAG(log_return,1) OVER (PARTITION BY ticker ORDER BY date) AS lag1,
       LAG(log_return,2) OVER (PARTITION BY ticker ORDER BY date) AS lag2,
       LEAD(log_return,1) OVER (PARTITION BY ticker ORDER BY date) AS r_tplus1  -- label candidate
FROM prices
WHERE date BETWEEN ? AND ?
ORDER BY ticker, date
LIMIT 20;
"""
df = pd.read_sql_query(sql, con, params=["2020-01-01","2020-06-30"])
df.head(10)

,ticker,date,r,lag1,lag2,r_tplus1
0,AAPL,2020-01-01,0.000000,NaN,NaN,0.002987
1,AAPL,2020-01-02,0.002987,0.000000,NaN,-0.002741
2,AAPL,2020-01-03,-0.002741,0.002987,0.000000,-0.008906
3,AAPL,2020-01-06,-0.008906,-0.002741,0.002987,-0.004547
4,AAPL,2020-01-07,-0.004547,-0.008906,-0.002741,-0.009916
5,AAPL,2020-01-08,-0.009916,-0.004547,-0.008906,0.000601
6,AAPL,2020-01-09,0.000601,-0.009916,-0.004547,0.013402
7,AAPL,2020-01-10,0.013402,0.000601,-0.009916,-0.004922
8,AAPL,2020-01-13,-0.004922,0.013402,0.000601,-0.006205
9,AAPL,2020-01-14,-0.006205,-0.004922,0.013402,0.004898



In SQLite (and most SQL dialects):

* The `WINDOW` clause must come **after** the `WHERE` clause, but **before** `ORDER BY`/`LIMIT`.

SQL clause order generally goes:

```
SELECT ...
FROM ...
WHERE ...
GROUP BY ...
HAVING ...
WINDOW ...
ORDER BY ...
LIMIT ...
```



In [8]:
sql = """
SELECT
  ticker, date, log_return AS r,
  AVG(log_return) OVER w AS roll_mean_20,
  AVG(log_return*log_return) OVER w
    - (AVG(log_return) OVER w)*(AVG(log_return) OVER w) AS roll_var_20
FROM prices
WHERE date BETWEEN ? AND ?
WINDOW w AS (
  PARTITION BY ticker
  ORDER BY date
  ROWS BETWEEN 19 PRECEDING AND CURRENT ROW
)

ORDER BY ticker, date
LIMIT 20;
"""
roll = pd.read_sql_query(sql, con, params=["2020-03-01","2020-06-30"])
roll.head(10)

,ticker,date,r,roll_mean_20,roll_var_20
0,AAPL,2020-03-02,0.000761,0.000761,0.000000
1,AAPL,2020-03-03,0.013588,0.007175,0.000041
2,AAPL,2020-03-04,-0.015471,-0.000374,0.000141
3,AAPL,2020-03-05,0.008594,0.001868,0.000121
4,AAPL,2020-03-06,0.001194,0.001733,0.000097
5,AAPL,2020-03-09,-0.006415,0.000375,0.000090
6,AAPL,2020-03-10,0.020004,0.003179,0.000124
7,AAPL,2020-03-11,0.007623,0.003735,0.000111
8,AAPL,2020-03-12,-0.011993,0.001987,0.000123
9,AAPL,2020-03-13,0.000745,0.001863,0.000111


`.clip(lower=0)` → forces negative values to 0 (variance should never be negative, but tiny floating-point errors can produce slightly negative numbers).
`.replace(0, pd.NA)` → avoids dividing by zero (if the std is zero, set it to NA).

In [9]:
# Addng two new features.
roll["roll_std_20"] = (roll["roll_var_20"].clip(lower=0)).pow(0.5)
roll["zscore_20"] = (roll["r"] - roll["roll_mean_20"]) / roll["roll_std_20"].replace(0, pd.NA)
roll.head(10)

,ticker,date,r,roll_mean_20,roll_var_20,roll_std_20,zscore_20
0,AAPL,2020-03-02,0.000761,0.000761,0.000000,0.000000,<NA>
1,AAPL,2020-03-03,0.013588,0.007175,0.000041,0.006413,1.0
2,AAPL,2020-03-04,-0.015471,-0.000374,0.000141,0.011891,-1.269691
3,AAPL,2020-03-05,0.008594,0.001868,0.000121,0.011005,0.611134
4,AAPL,2020-03-06,0.001194,0.001733,0.000097,0.009847,-0.054794
5,AAPL,2020-03-09,-0.006415,0.000375,0.000090,0.009488,-0.715601
6,AAPL,2020-03-10,0.020004,0.003179,0.000124,0.011151,1.508813
7,AAPL,2020-03-11,0.007623,0.003735,0.000111,0.010534,0.369085
8,AAPL,2020-03-12,-0.011993,0.001987,0.000123,0.011093,-1.260216
9,AAPL,2020-03-13,0.000745,0.001863,0.000111,0.010531,-0.106149


The **`WITH` clause** in SQL introduces what’s called a **Common Table Expression (CTE)**.


A **CTE** is like creating a temporary, named result set (a “virtual table”) that you can use inside your main query. It only exists for the duration of that query.

Syntax:

```sql
WITH cte_name AS (
  SELECT ...
  FROM ...
  WHERE ...
)
SELECT *
FROM cte_name
WHERE ...
```

Think of it as: *“Run this subquery once, give it a name, and then treat it like a table in the following query.”*

**`ROW_NUMBER() OVER()`**, since it’s one of the most common SQL window functions.



* `ROW_NUMBER()` assigns a **unique sequential number** to each row in the result set.
* The numbering starts at 1 for each partition (or the whole dataset if no partition).
* The **`OVER(...)`** clause tells SQL how to order and group the rows before numbering.

---

### 2. General syntax

```sql
ROW_NUMBER() OVER (
    PARTITION BY column1, column2, ...
    ORDER BY columnX [ASC|DESC]
)
```

* **`PARTITION BY`** → splits rows into groups, and the numbering restarts in each group.
* **`ORDER BY`** → determines the order of rows inside each group (decides which row is 1, 2, 3, …).

---

### 3. Example without partition

```sql
SELECT name, salary,
       ROW_NUMBER() OVER (ORDER BY salary DESC) AS rank
FROM employees;
```

If salaries were `[100k, 90k, 80k]`, the output would be:

| name | salary | rank |
| ---- | ------ | ---- |
| A    | 100k   | 1    |
| B    | 90k    | 2    |
| C    | 80k    | 3    |

---

### 4. Example with partition

```sql
SELECT department, name, salary,
       ROW_NUMBER() OVER (
         PARTITION BY department
         ORDER BY salary DESC
       ) AS rank
FROM employees;
```

Now the row numbers reset within each department. So you get the **top earner per department** ranked as 1.



```sql
ROW_NUMBER() OVER (
  PARTITION BY ticker
  ORDER BY ABS(log_return) DESC
) AS rn
```

* **Partition by `ticker`** → each stock ticker is ranked independently.
* **Order by `ABS(log_return) DESC`** → biggest absolute return is rank 1, second-biggest is 2, etc.
* `AS rn` → gives the column alias `rn`.

So if `AAPL` had returns `[-0.12, 0.09, 0.07, 0.01]`, the ranks would be:

| log\_return | abs | rn |
| ----------- | --- | -- |
| -0.12       | .12 | 1  |
| 0.09        | .09 | 2  |
| 0.07        | .07 | 3  |
| 0.01        | .01 | 4  |




In [10]:
sql = """
WITH ranked AS (
  SELECT
    ticker, date, log_return,
    ABS(log_return) AS abs_move,
    ROW_NUMBER() OVER (
      PARTITION BY ticker
      ORDER BY ABS(log_return) DESC
    ) AS rn
  FROM prices
  WHERE date BETWEEN ? AND ?
)
SELECT * FROM ranked WHERE rn <= 3
ORDER BY ticker, rn;
"""
topk = pd.read_sql_query(sql, con, params=["2020-01-01","2020-08-01"])
topk.head(15)

,ticker,date,log_return,abs_move,rn
0,AAPL,2020-02-06,-0.025168,0.025168,1
1,AAPL,2020-05-25,0.022448,0.022448,2
2,AAPL,2020-06-25,-0.021286,0.021286,3
3,AMZN,2020-04-22,-0.027604,0.027604,1
4,AMZN,2020-02-24,-0.025325,0.025325,2
5,AMZN,2020-04-13,-0.022550,0.022550,3
6,BAC,2020-07-09,0.024016,0.024016,1
7,BAC,2020-06-30,0.021946,0.021946,2
8,BAC,2020-01-15,-0.020479,0.020479,3
9,CSCO,2020-07-03,-0.030028,0.030028,1
